In [192]:
#Update your token
STUDENT_TOKEN = 'EUGENIO MARCHIORI'

## ignore this code, just used for submission
import requests
import pprint
import json
import random
import math
from copy import copy, deepcopy

def get_problem(problem_id):
  r = requests.get('https://emarchiori.eu.pythonanywhere.com/get-problem?TOKEN=%s&problem=%s' % (STUDENT_TOKEN, problem_id))
  if not r.status_code == 200:
    print('\033[91m' + str(r.status_code))
  return r.json()

def submit_answer(problem_id, answer):
  r = requests.get('https://emarchiori.eu.pythonanywhere.com/submit-answer?TOKEN=%s&problem=%s' % (STUDENT_TOKEN, problem_id), json = answer)
  if not r.status_code == 200:
    print('\033[91m' + str(r.status_code))
  result = r.json()['result']
  if result == 'PASSED':
    print('\033[92m' + result)
  else:
    print('\033[91m' + result)

In [193]:
def format_value(value):
  if value == '.':
    return '..'
  else:
    s = str(value)
    if len(s) == 1:
      s = ' ' + s
    return s

def format_h(const, max):
  s = ''
  for i in range(max - len(const)):
    s = s + '   '
  s = s + ' '.join(map(format_value, const))
  return s

def pretty_print(board, h_const, v_const):
  max_h = 1
  for c in h_const:
    max_h = max(max_h, len(c))
  max_v = 1
  for c in v_const:
    max_v = max(max_v, len(c))

  h_offset = ' '.join(map(lambda x: '  ', range(max_h)))
  for i in range(max_v):
    print('    ' + h_offset + ' '.join(map(lambda x: '  ' if i >= len(x) else format_value(x[i]), v_const)))
  for r in range(len(board)):
    row = board[r]
    print((h_offset if r ==0 else format_h(h_const[r-1], max_h)) + ' ' +  ' '.join(map(format_value, row)))

In [194]:
problem = get_problem('kakuro')

pprint.pprint(problem)

board = problem['board']
h_const = problem['h_constraints']
v_const = problem['v_constraints']

print('Input puzzle')
pretty_print(board, h_const, v_const)

# This does not solve the problem, just makes it the same as the input
solution = board

# We print the solution in a way to make it easy to debug and check
print('Solution')
pretty_print(solution, h_const, v_const)

answer = {
    'board': board,
    'h_const': h_const,
    'v_const': v_const,
    'id': problem['id']
}

# The answer is submitted to the server, this is just to check the format (it does not check the solution itself)
# Note: We are doing this just to get used to it in following exercises mostly
submit_answer('kakuro', answer)

{'board': [['X', 'X', 'X', 'X', 'X'],
           ['X', '.', '.', 'X', 'X'],
           ['X', '.', '.', '.', 'X'],
           ['X', 'X', '.', '.', '.'],
           ['X', 'X', 'X', '.', '.']],
 'h_constraints': [[11], [14], [20], [17]],
 'id': 3,
 'v_constraints': [[16], [7], [23], [16]]}
Input puzzle
      16  7 23 16
    X  X  X  X  X
11  X .. ..  X  X
14  X .. .. ..  X
20  X  X .. .. ..
17  X  X  X .. ..
Solution
      16  7 23 16
    X  X  X  X  X
11  X .. ..  X  X
14  X .. .. ..  X
20  X  X .. .. ..
17  X  X  X .. ..
Ignore: not checking values yet


In [195]:
# representation
class Kakuro:
    def __init__(self, puzzle):
        self.board = puzzle['board']
        self.h_constraints = puzzle['h_constraints']
        self.v_constraints = puzzle['v_constraints']
        self.id = puzzle.get('id', None)

    def pretty_print(self):
        max_h = 1
        for c in self.h_constraints:
            max_h = max(max_h, len(c))
        max_v = 1
        for c in self.v_constraints:
            max_v = max(max_v, len(c))

        h_offset = ' '.join(map(lambda x: '  ', range(max_h)))
        for i in range(max_v):
            print('    ' + h_offset + ' '.join(map(lambda x: '  ' if i >= len(x) else format_value(x[i]), self.v_constraints)))
        for r in range(len(self.board)):
            row = self.board[r]
            print((h_offset if r ==0 else format_h(self.h_constraints[r-1], max_h)) + ' ' +  ' '.join(map(format_value, row)))

    def is_valid(self):
        # Check horizontal constraints
        for r, row in enumerate(self.board):
            for c, value in enumerate(row):
                if value != 'X' and value != '.':
                    # Check the sum of horizontal "word"
                    if not self.check_horizontal(r, c):
                        return False
                    # Check the sum of vertical "word"
                    if not self.check_vertical(r, c):
                        return False
        return True
    
    def check_horizontal(self, row, col):
        target_sum = self.h_constraints[row-1][0]
        total, count = 0, 0
        for c in range(col, len(self.board[row])):
            cell = self.board[row][c]
            if cell == 'X':  # End of the horizontal "word"
                break
            if cell == '.':  # Empty cell, not yet assigned
                continue
            total += int(cell)
            count += 1
        if count == 0:  # No cells filled in yet
            return True
        if total > target_sum:
            return False  # Sum exceeded
        if 'X' not in self.board[row][col:] and total != target_sum:
            return False  # Reached end of "word" and sum does not match
        return True

    def check_vertical(self, row, col):
        target_sum = self.v_constraints[col-1][0]
        total, count = 0, 0
        for r in range(row, len(self.board)):
            cell = self.board[r][col]
            if cell == 'X':  # End of the vertical "word"
                break
            if cell == '.':  # Empty cell, not yet assigned
                continue
            total += int(cell)
            count += 1
        if count == 0:  # No cells filled in yet
            return True
        if total > target_sum:
            return False  # Sum exceeded
        if 'X' not in [self.board[r][col] for r in range(row, len(self.board))] and total != target_sum:
            return False  # Reached end of "word" and sum does not match
        return True




In [196]:
import itertools

import itertools


import itertools

import itertools

import itertools

def propagate_word_constraint(empty_indices, filled_cells, target_sum):
    """
    Determine the possible values for each empty cell in a "word" by considering all other cells.
    """
    remaining_sum = target_sum - sum(filled_cells)
    
    # Exclude numbers already used in the filled cells when generating permutations
    available_numbers = [num for num in range(1, 10) if num not in filled_cells]
    
    # Generate all possible permutations of numbers that can fill the empty cells and match the remaining sum.
    valid_permutations = [seq for seq in itertools.permutations(available_numbers, len(empty_indices))
                          if sum(seq) == remaining_sum and all(num not in filled_cells for num in seq)]
    
    # For each cell position, determine the possible values it can take
    possible_values_per_cell = [set() for _ in empty_indices]
    for perm in valid_permutations:
        for i, val in enumerate(perm):
            possible_values_per_cell[i].add(val)
    
    return possible_values_per_cell



def propagate_constraints(board, row, col, h_constraints, v_constraints):
    # Find the "words" affected by the placement of a number at (row, col)
    h_word, h_target_sum = find_word_and_sum(board, row, col, h_constraints, True)
    v_word, v_target_sum = find_word_and_sum(board, row, col, v_constraints, False)

    # Initialize possibilities for the horizontal and vertical words
    h_word_possibilities = [set(range(1, 10)) if x == '.' else {int(x)} for x in h_word]
    v_word_possibilities = [set(range(1, 10)) if x == '.' else {int(x)} for x in v_word]

    # Now only call propagate_word_constraint for empty cells
    if h_target_sum:
        h_empty_indices = [i for i, x in enumerate(h_word) if x == '.']
        h_filled_cells = [int(x) for x in h_word if x != '.']
        h_word_possibilities_for_empty = propagate_word_constraint(h_empty_indices, h_filled_cells, h_target_sum)
        for i, index in enumerate(h_empty_indices):
            h_word_possibilities[index] = h_word_possibilities_for_empty[i]

    if v_target_sum:
        v_empty_indices = [i for i, x in enumerate(v_word) if x == '.']
        v_filled_cells = [int(x) for x in v_word if x != '.']
        v_word_possibilities_for_empty = propagate_word_constraint(v_empty_indices, v_filled_cells, v_target_sum)
        for i, index in enumerate(v_empty_indices):
            v_word_possibilities[index] = v_word_possibilities_for_empty[i]

    # Get the index of the current cell in the horizontal and vertical words
    h_index = h_word.index('.') if '.' in h_word else None
    v_index = v_word.index('.') if '.' in v_word else None

    # Combine the possibilities for the current cell from the horizontal and vertical "words"
    combined_possibilities = set(range(1, 10))
    if h_index is not None and h_index < len(h_word_possibilities):
        combined_possibilities.intersection_update(h_word_possibilities[h_index])
    if v_index is not None and v_index < len(v_word_possibilities):
        combined_possibilities.intersection_update(v_word_possibilities[v_index])

    # Return only the combined possibilities for the current cell
    return combined_possibilities






def find_word_and_sum(board, row, col, constraints, is_horizontal):
    """
    Given a board, a cell position (row, col), and constraints, this function
    will return the "word" (a sequence of values and empty cells) that the cell
    belongs to and the associated target sum for that "word".
    """
    if is_horizontal:
        start, end = col, col
        while start > 0 and board[row][start - 1] != 'X':
            start -= 1
        while end < len(board[row]) and board[row][end] != 'X':
            end += 1
        word = board[row][start:end]
        target_sum = constraints[row - 1][0] if row > 0 else None
    else:
        start, end = row, row
        while start > 0 and board[start - 1][col] != 'X':
            start -= 1
        while end < len(board) and board[end][col] != 'X':
            end += 1
        word = [board[i][col] for i in range(start, end)]
        target_sum = constraints[col - 1][0] if col > 0 else None

    return word, target_sum




In [197]:
# Let's add debugging print statements to both the get_possible_values and the solve_kakuro functions.

def get_possible_values(board, row, col, h_constraints, v_constraints):
    possible_values = set(range(1, 10))
    # Propagate constraints from the current board state
    combined_possibilities = propagate_constraints(board, row, col, h_constraints, v_constraints)
    # The combined possibilities are for the current cell only
    possible_values.intersection_update(combined_possibilities)
    return possible_values

def calculate_mrv(board, h_constraints, v_constraints):
    mrv_cells = []
    for row in range(len(board)):
        for col in range(len(board[0])):
            if board[row][col] == '.':
                possible_values = get_possible_values(board, row, col, h_constraints, v_constraints)
                mrv_cells.append(((row, col), len(possible_values)))
                # Debug print to show the cell and the number of possible values
                print(f"Cell ({row}, {col}) has {len(possible_values)} possible values.")
    # Sort by the number of possible values, which is the second item in the tuples
    mrv_cells.sort(key=lambda item: item[1])
    # Debug print to show the sorted MRV cells
    print(f"MRV cells sorted by possible values: {mrv_cells}")
    return mrv_cells



def solve_kakuro(board, h_constraints, v_constraints):
    print("Starting to solve Kakuro...")
    backtracks = 0  # To count the number of backtracks

    # Initialize MRV with the first call to calculate_mrv
    mrv_cells = calculate_mrv(board, h_constraints, v_constraints)
    if not mrv_cells:
        print("No empty cells found, puzzle is already solved or invalid.")
        return board, backtracks

    # Start with the cell with the minimum remaining values
    (row, col), _ = mrv_cells[0]
    print(f"Starting with cell ({row}, {col}) with minimum remaining values.")
    stack = [(board, row, col)]

    while stack:
        board, row, col = stack.pop()  # Get the current state and cell to fill
        print(f"\nCurrent state: {board}")
        print(f"Current position: ({row}, {col})")
        print(f"Stack size: {len(stack)}, Backtracks: {backtracks}")

        # Get possible values for the current cell
        possible_values = get_possible_values(board, row, col, h_constraints, v_constraints)

        # If there are no possible values, we need to backtrack
        if not possible_values:
            backtracks += 1
            print("No possible values, backtracking...")
            continue  # Backtrack to the previous state

        # Try each possible value and push the new state onto the stack
        for value in possible_values:
            new_board = [list(r) for r in board]  # Create a deep copy of the board
            new_board[row][col] = str(value)  # Assign the value as a string
            print(f"Trying value {value} at cell ({row}, {col})")

            # Update MRV after the assignment
            next_mrv_cells = calculate_mrv(new_board, h_constraints, v_constraints)
            if next_mrv_cells:
                next_row, next_col = next_mrv_cells[0][0]
                print(f"Next cell to try is ({next_row}, {next_col}) with MRV.")
                stack.append((new_board, next_row, next_col))  # Push the new state with MRV cell
            else:
                # If MRV list is empty, it means all cells are filled
                print("Solution found!")
                return new_board, backtracks

    # If the stack is empty, no solution was found
    print("No solution found.")
    return None, backtracks




In [210]:
problem = get_problem('kakuro')
 

# print the initial board
pretty_print(problem['board'], problem['h_constraints'], problem['v_constraints'])

# Now, let's solve the puzzle again with the refined 'get_possible_values' function
solution, backtracks = solve_kakuro(problem['board'], problem['h_constraints'], problem['v_constraints'])
solution, backtracks

# print the solution
pretty_print(solution, problem['h_constraints'], problem['v_constraints'])

      23  7  6  7
    X  X  X  X  X
 5  X  X  X .. ..
13  X .. .. .. ..
14  X .. .. .. ..
11  X .. ..  X  X
Starting to solve Kakuro...
Cell (1, 3) has 3 possible values.
Cell (1, 4) has 3 possible values.
Cell (2, 1) has 1 possible values.
Cell (2, 2) has 3 possible values.
Cell (2, 3) has 3 possible values.
Cell (2, 4) has 3 possible values.
Cell (3, 1) has 2 possible values.
Cell (3, 2) has 3 possible values.
Cell (3, 3) has 3 possible values.
Cell (3, 4) has 3 possible values.
Cell (4, 1) has 3 possible values.
Cell (4, 2) has 2 possible values.
MRV cells sorted by possible values: [((2, 1), 1), ((3, 1), 2), ((4, 2), 2), ((1, 3), 3), ((1, 4), 3), ((2, 2), 3), ((2, 3), 3), ((2, 4), 3), ((3, 2), 3), ((3, 3), 3), ((3, 4), 3), ((4, 1), 3)]
Starting with cell (2, 1) with minimum remaining values.

Current state: [['X', 'X', 'X', 'X', 'X'], ['X', 'X', 'X', '.', '.'], ['X', '.', '.', '.', '.'], ['X', '.', '.', '.', '.'], ['X', '.', '.', 'X', 'X']]
Current position: (2, 1)
Stack size: 0, B